<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Activity" data-toc-modified-id="Activity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Activity</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Setting-up-the-data" data-toc-modified-id="Setting-up-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up the data</a></span></li><li><span><a href="#Modeling-changes-to-a-tissue-during-infection" data-toc-modified-id="Modeling-changes-to-a-tissue-during-infection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling changes to a tissue during infection</a></span></li><li><span><a href="#Run-simulation-and-visualize" data-toc-modified-id="Run-simulation-and-visualize-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run simulation and visualize</a></span></li><li><span><a href="#How-immune-cells-affect-infection" data-toc-modified-id="How-immune-cells-affect-infection-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>How immune cells affect infection</a></span></li><li><span><a href="#Plot-infection-with-immune-system-over-time" data-toc-modified-id="Plot-infection-with-immune-system-over-time-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Plot infection with immune system over time</a></span></li></ul></div>

# Activity

    
    Imagine a tissue with n x n cells with different states:
    
    H: Healthy
    VA: Viral assembly
    VR: Viral release
    D: Dead
    
    INFECT_PROB = probability of infection (i.e. the likelihood of a cell becoming infected)
    
    

In [2]:
# import sys
# !{sys.executable} -m pip install plotly

distributed 1.21.8 requires msgpack, which is not installed.
jupyterlab-server 1.0.0 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
You are using pip version 10.0.1, however version 21.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Libraries

In [1]:
#Importing libraries: computer code 'cookbooks'

import matplotlib.pyplot
import pandas as pd
import numpy as np
import random
import plotly as py
import plotly.graph_objs as go

# Setting up the data

In [2]:
def setUpData(n, max_starting_infected, infect_prob):
    H_d = {}
    VA_d = {}
    VR_d = {}
    D_d = {}

    n_starting_infected = random.randint(0, max_starting_infected) # random number generated for of starting number infected cells

    # choose (x,y) where to place the infected cells in tissue
    points = set()
    while len(points) < n_starting_infected:
        points.add((random.randint(0, n-1), random.randint(0, n-1))) 
    for i in range(n):
        for j in range(n):
            if (i, j) in points:
                VR_d[(i, j)] ={"infectProb": infect_prob}
            else:
                H_d[(i, j)] = {}
    print("# starting infected cells: {}".format(n_starting_infected))
    return H_d, VA_d, VR_d, D_d

def assignTissue(n, H_d, VA_d, VR_d, D_d):
    tissue = pd.DataFrame("-", index = range(n), columns = range(n))
    for cord in H_d.keys():
        tissue.iloc[cord[0], cord[1]] = "H"
    for cord in VA_d.keys():
        tissue.iloc[cord[0], cord[1]] = "VA"
    for cord in VR_d.keys():
        tissue.iloc[cord[0], cord[1]] = "VR"
    for cord in D_d.keys():
        tissue.iloc[cord[0], cord[1]] = "D"
    return tissue



INFECT_PROB = 0.9 
n = 5 # size of tissue
max_starting_infected = 5 # you choose maximum number of starting infected cells
    
H_d, VA_d, VR_d, D_d = setUpData(n, max_starting_infected, INFECT_PROB)      
tissue = assignTissue(n, H_d, VA_d, VR_d, D_d) 
tissue


# starting infected cells: 3


,0,1,2,3,4
0,H,VR,H,H,H
1,H,H,H,H,H
2,H,H,VR,VR,H
3,H,H,H,H,H
4,H,H,H,H,H


# Modeling changes to a tissue during infection

    Next stages for:
    
        Healthy cell: 
            If adjacent to cells releasing viruses (i.e. "VR"), give a random probability between 0 and 1.
            If probability <= INFECT_PROB, healthy cell is infected and becomes H -> VA
            
        Viral assembly:
            Release of viruses (i.e. VA -> VR)
            
        Viral release:
            Dies (i.e. VR -> D)
            
        Dead cell:
            No change
            
            

In [9]:


def one_time_step(H_d, VA_d, VR_d, D_d):                      #Q2: how many times does this run
    
#     updated_tissue = tissue.copy()
#     for row in range(n):
#         for col in range(n):
            
    H_d_updated = H_d.copy()
    VA_d_updated = VA_d.copy()
    VR_d_updated = VR_d.copy()
    D_d_updated = D_d.copy()
    
    for VR in VR_d.keys():
        row = VR[0]
        col = VR[1]
        infect_prob = VR_d[VR]["infectProb"]
        if tuple((row+1, col)) in H_d_updated:
             if random.random() < infect_prob:
                del H_d_updated[(row+1, col)]
                VA_d_updated[(row+1, col)] = {"infectProb": infect_prob}
                
        if tuple((row+1, col+1)) in H_d_updated:
             if random.random() < infect_prob:
                del H_d_updated[(row+1, col+1)]
                VA_d_updated[(row+1, col+1)] = {"infectProb": infect_prob}
                
        if tuple((row, col+1)) in H_d_updated:
             if random.random() < infect_prob:
                del H_d_updated[(row, col+1)]
                VA_d_updated[(row, col+1)] = {"infectProb": infect_prob}
                
        if tuple((row-1, col)) in H_d_updated:
             if random.random() < infect_prob:
                del H_d_updated[(row-1, col)]
                VA_d_updated[(row-1, col)] = {"infectProb": infect_prob}
                
        if tuple((row, col-1)) in H_d_updated:
             if random.random() < infect_prob:
                del H_d_updated[tuple((row, col-1))]
                VA_d_updated[(row, col-1)] = {"infectProb": infect_prob}
                
        if tuple((row-1, col-1)) in H_d_updated:
             if random.random() < infect_prob:
                del H_d_updated[tuple((row-1, col-1))]
                VA_d_updated[(row-1, col-1)] = {"infectProb": infect_prob}
          
    for VA in VA_d.keys():
        del VA_d_updated[VA]
        VR_d_updated[VA] = VA_d[VA]
    for VR in VR_d.keys():
        del VR_d_updated[VR]
        D_d_updated[VR] = VR_d[VR]
    return H_d_updated, VA_d_updated, VR_d_updated, D_d_updated





            
def t_time_step(t, n, H_d, VA_d, VR_d, D_d, infect_prob):                    #Q2 When you call this function how many times does above run
    
    tissue = assignTissue(n, H_d, VA_d, VR_d, D_d) 
    
    out_tissue = []
    out_H = [H_d]
    out_VA = [VA_d]
    out_VR = [VR_d]
    out_D = [D_d]
    
    out_tissue.append(tissue.values)
    
    #updated_tissue = tissue.copy()
    for step in range(t):
        H_d, VA_d, VR_d, D_d = one_time_step(H_d, VA_d, VR_d, D_d)
        updated_tissue = assignTissue(n, H_d, VA_d, VR_d, D_d) 
        out_tissue.append(updated_tissue.values)
        out_H.append(H_d)
        out_VA.append(VA_d)
        out_VR.append(VR_d)
        out_D.append(D_d)
        
    return out_tissue , out_H,out_VA, out_VR, out_D


INFECT_PROB = 0.9 
n = 5 # size of tissue
max_starting_infected = 5 
t = 3
    
H_d, VA_d, VR_d, D_d = setUpData(n, max_starting_infected, INFECT_PROB)      
tissue_frames, H_steps, VA_steps, VR_steps, D_steps = t_time_step(t, n, H_d, VA_d, VR_d, D_d, INFECT_PROB)
print(tissue_frames)
for h in H_steps:
    print(h)


# starting infected cells: 3
[array([['H', 'VR', 'H', 'H', 'H'],
       ['H', 'H', 'H', 'H', 'H'],
       ['H', 'H', 'H', 'H', 'H'],
       ['H', 'H', 'VR', 'H', 'H'],
       ['VR', 'H', 'H', 'H', 'H']], dtype=object), array([['VA', 'D', 'VA', 'H', 'H'],
       ['H', 'VA', 'VA', 'H', 'H'],
       ['H', 'VA', 'VA', 'H', 'H'],
       ['VA', 'VA', 'D', 'VA', 'H'],
       ['D', 'VA', 'VA', 'VA', 'H']], dtype=object), array([['VR', 'D', 'VR', 'H', 'H'],
       ['H', 'VR', 'VR', 'H', 'H'],
       ['H', 'VR', 'VR', 'H', 'H'],
       ['VR', 'VR', 'D', 'VR', 'H'],
       ['D', 'VR', 'VR', 'VR', 'H']], dtype=object), array([['D', 'D', 'D', 'VA', 'H'],
       ['VA', 'D', 'D', 'VA', 'H'],
       ['VA', 'D', 'D', 'VA', 'H'],
       ['D', 'D', 'D', 'D', 'VA'],
       ['D', 'D', 'D', 'D', 'VA']], dtype=object)]
{(0, 0): {}, (0, 2): {}, (0, 3): {}, (0, 4): {}, (1, 0): {}, (1, 1): {}, (1, 2): {}, (1, 3): {}, (1, 4): {}, (2, 0): {}, (2, 1): {}, (2, 2): {}, (2, 3): {}, (2, 4): {}, (3, 0): {}, (3, 1): {},

# Run simulation and visualize

In [10]:
INFECT_PROB = 0.5
n = 25 # size of tissue
max_starting_infected = 5 
t = 20
    
H_d, VA_d, VR_d, D_d = setUpData(n, max_starting_infected, INFECT_PROB)      
t_time_step(t, n, H_d, VA_d, VR_d, D_d, INFECT_PROB)

tissue_frames, H_steps, VA_steps, VR_steps, D_steps =t_time_step(t, n, H_d, VA_d, VR_d, D_d, INFECT_PROB)





# starting infected cells: 1


In [11]:
#Plot simulation - do not worry about what is happening in code here, just what movie it makes
numerify = {'H': 0, 'VA': 0.25, 'VR': 0.5, 'D': 0.75}

legend_colors = [
    [0, "#56B4E9"],
    [0.25,  "#56B4E9"],
    [0.25, "#F0E442"],
    [0.5, "#F0E442"],
    [0.5, "#D55E00"],
    [0.75, "#D55E00"],
    [0.75, "#000000"],
    [1.0, "#000000"]
]

numbers = [0, 0.25, 0.5, 0.75]
names   = ['Healthy', 'Viral Assembly', 'Viral Release', 'Dead']


# Numerify the cells for plotting
tissue_frames_numeric = []
for t in range(len(tissue_frames)):
    tissue_frames_numeric.append(tissue_frames[t].copy())
    for i in range(tissue_frames[t].shape[0]):
        for j in range(tissue_frames[t].shape[1]):
            tissue_frames_numeric[t][i][j] = numerify[tissue_frames[t][i][j]]
            
fig = go.Figure(data=go.Heatmap(z=tissue_frames_numeric[0], colorscale=legend_colors, colorbar=dict(tickvals=numbers, ticktext=names, thickness=25)),
               frames=[go.Frame(data=go.Heatmap(z=tissue_frames_numeric[i])) for i in range(len(tissue_frames_numeric))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()